In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st

In [ ]:
homer_path = ''
# homer_path = 'export PATH=$PATH:/sc/projects/sci-kretzmer/chair/software/tools/homer/bin/;'
env = ''
# env = 'eval "$(conda shell.bash hook)";conda activate /sc/projects/sci-kretzmer/chair/software/conda/metilene3;export MPLBACKEND=qtagg;'

In [ ]:
os.system(env+'python ../metilene3/metilene3.py \
    -i ./data/GSE186458_blood.input.tsv \
    -o ./Blood \
    -t 16 \
    -plot True\
')

In [ ]:
rename_dict = {
    'P2_2_3_3_3_3': 'Mono+Gran(hypo) vs B+NK+T+naiveT',
    'N0_0_0_0_2_3': 'T vs naiveT(hypo)',
}

In [ ]:
dmrs = pd.read_table('./Blood/DMRs.tsv').sort_values(['chr','start','stop'])
    
dmrs['strand'] = '+'
dmrs['stop_1'] = dmrs['stop']+1

path = './Blood/motif/'
os.system('mkdir '+path)
for i in ['P2|2|3|3|3|3', 'N0|0|0|0|2|3']:
    dmrs.loc[(dmrs['meandiffabs']>=0.5)&(dmrs['DMTree'].fillna('').str.contains(i.replace('|','\|')+','))][['chr','start','stop_1','strand']].\
        to_csv('./Blood/motif/'+i.replace('|','_')+'.bed', sep='\t', header=False)
    dmrs.loc[(dmrs['meandiffabs']>=0.5)&(~dmrs['DMTree'].fillna('').str.contains(i.replace('|','\|')+','))][['chr','start','stop_1','strand']].\
        to_csv('./Blood/motif/'+i.replace('|','_')+'.anti.bed', sep='\t', header=False)
    
    os.system(env+homer_path+'findMotifsGenome.pl ./Blood/motif/'+i.replace('|','_')+'.bed'+\
                  ' hg19'+\
                  ' ./Blood/motif/'+i.replace('|','_')+' -bits -size 250  -bg ./Blood/motif/'+i.replace('|','_')+'.anti.bed &')